In [88]:
!pip install pandas gdown scikit-learn ultralytics

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import gdown
import os
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from ultralytics import YOLO

##Changes here in v2
## Use this code to download the testing folder. You must include these 2 lines without any changes
test_folder_id = '1LZIhiV9l82W4fNpfaBHyGvh8XBX6SoU2'
gdown.download_folder(id=test_folder_id, output="test_folder", quiet=False, use_cookies=False)


Retrieving folder contents


Retrieving folder 1qO-cQtRdp-hLu6LMQ68OEv0zA791puJO test_images
Processing file 1cxftTp4A1_13gWe4QeAlhK9AcGzIDBEH Kek Lapis.jpg
Processing file 1aqVkDJiANm0MqLeTJALC1RuKZgt0IlI4 Kuih Kaswi Pandan.jpg
Processing file 11K7Hc9nt3Qllw4lG4ajxineVHvXKmuY6 Kuih Ketayap.jpg
Processing file 1qTcPAX5eSPLkAkhlyGfgooDXHZPFoJOM Kuih Lapis.jpg
Processing file 17CPxIkfU_tecxiF1-G4nXTD0Vv_1biB- Kuih Seri Muka.png
Processing file 1MEhhjAoBK66wtj6Xs1orsrA2vR9qLjhR Kuih Talam.jpg
Processing file 1KmI2JRpz4DnCAmPbCWoK31BJBH4ax8i7 Kuih Ubi Kayu.jpg
Processing file 1QFZdtM4ok-vH0kPgA7gdZQzT1aSppWcF Onde-Onde.jpg
Processing file 12ufiwhl6whVp33F05RLiDRieRSN13XlP test_labels.txt


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1cxftTp4A1_13gWe4QeAlhK9AcGzIDBEH
To: c:\Users\zanyi\Documents\GitHub\NAIC\submission\test_folder\test_images\Kek Lapis.jpg
100%|██████████| 1.27M/1.27M [00:00<00:00, 2.19MB/s]
Downloading...
From: https://drive.google.com/uc?id=1aqVkDJiANm0MqLeTJALC1RuKZgt0IlI4
To: c:\Users\zanyi\Documents\GitHub\NAIC\submission\test_folder\test_images\Kuih Kaswi Pandan.jpg
100%|██████████| 418k/418k [00:00<00:00, 1.65MB/s]
Downloading...
From: https://drive.google.com/uc?id=11K7Hc9nt3Qllw4lG4ajxineVHvXKmuY6
To: c:\Users\zanyi\Documents\GitHub\NAIC\submission\test_folder\test_images\Kuih Ketayap.jpg
100%|██████████| 373k/373k [00:00<00:00, 1.92MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qTcPAX5eSPLkAkhlyGfgooDXHZPFoJOM
To: c:\Users\zanyi\Documents\GitHub\NAIC\submission\test_folder\test_images\Kuih Lapis.jpg
100%|██████████| 56.6k/5

['test_folder\\test_images\\Kek Lapis.jpg',
 'test_folder\\test_images\\Kuih Kaswi Pandan.jpg',
 'test_folder\\test_images\\Kuih Ketayap.jpg',
 'test_folder\\test_images\\Kuih Lapis.jpg',
 'test_folder\\test_images\\Kuih Seri Muka.png',
 'test_folder\\test_images\\Kuih Talam.jpg',
 'test_folder\\test_images\\Kuih Ubi Kayu.jpg',
 'test_folder\\test_images\\Onde-Onde.jpg',
 'test_folder\\test_labels.txt']

In [79]:
model_folder_id = "1XK3T_-hU5WLMcqlrLZe1zWOZkbiGU1tZ"
gdown.download_folder(id=model_folder_id, output="model", quiet=False, use_cookies=False)

Retrieving folder contents


Processing file 1_7BS5j_Q45__GDzONhUppopYOuzLJi1d best.pt
Processing file 1bQ3S0yogD5SMnEqWq-Ggzw9YsVKXKmYH labels.txt


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1_7BS5j_Q45__GDzONhUppopYOuzLJi1d
To: c:\Users\zanyi\Documents\GitHub\NAIC\submission\model\best.pt
100%|██████████| 20.9M/20.9M [00:07<00:00, 2.88MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bQ3S0yogD5SMnEqWq-Ggzw9YsVKXKmYH
To: c:\Users\zanyi\Documents\GitHub\NAIC\submission\model\labels.txt
100%|██████████| 138/138 [00:00<00:00, 138kB/s]
Download completed


['model\\best.pt', 'model\\labels.txt']

In [ ]:
model = YOLO("model/best.pt") #load model

In [81]:
# get labels
label_map = {}
with open("model/labels.txt", "r", encoding="utf-8") as f:
    for _, line in enumerate(f):
        parts = line.split(" ", 1)

        idx, label = parts
        label_map[int(idx)] = label.strip()
reverse_labels = {v.replace("-", " "): k for k, v in label_map.items()}

print(label_map)
print(reverse_labels)

{0: 'Kuih Talam', 1: 'Kuih Lapis', 2: 'Kuih Ubi Kayu', 3: 'Onde-Onde', 4: 'Kuih Kaswi Pandan', 5: 'Kek Lapis', 6: 'Kuih Seri Muka', 7: 'Kuih Ketayap'}
{'Kuih Talam': 0, 'Kuih Lapis': 1, 'Kuih Ubi Kayu': 2, 'Onde Onde': 3, 'Kuih Kaswi Pandan': 4, 'Kek Lapis': 5, 'Kuih Seri Muka': 6, 'Kuih Ketayap': 7}


In [82]:
#test the data
folderPath = r"test_folder\test_images"
store = []
pred_label_indexes_store = []
for path in os.listdir(folderPath):
    filePath = os.path.join(folderPath,path)
    prediction = model.predict(filePath)
    logits = prediction[0].names[prediction[0].probs.top1]
    pred_label_indexes = reverse_labels[logits]
    print(f"prediction of this is: {logits}")
    store.append([path, logits])
    pred_label_indexes_store.append(pred_label_indexes)



image 1/1 c:\Users\zanyi\Documents\GitHub\NAIC\submission\test_folder\test_images\Kek Lapis.jpg: 640x640 Kek Lapis 1.00, Kuih Seri Muka 0.00, Kuih Ubi Kayu 0.00, Kuih Lapis 0.00, Kuih Ketayap 0.00, 16.0ms
Speed: 43.6ms preprocess, 16.0ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)
prediction of this is: Kek Lapis

image 1/1 c:\Users\zanyi\Documents\GitHub\NAIC\submission\test_folder\test_images\Kuih Kaswi Pandan.jpg: 640x640 Kuih Kaswi Pandan 0.66, Kuih Lapis 0.31, Kuih Seri Muka 0.02, Kuih Ubi Kayu 0.01, Kuih Ketayap 0.00, 15.8ms
Speed: 35.2ms preprocess, 15.8ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)
prediction of this is: Kuih Kaswi Pandan

image 1/1 c:\Users\zanyi\Documents\GitHub\NAIC\submission\test_folder\test_images\Kuih Ketayap.jpg: 640x640 Kuih Ketayap 1.00, Kuih Ubi Kayu 0.00, Kek Lapis 0.00, Kuih Lapis 0.00, Kuih Seri Muka 0.00, 15.5ms
Speed: 68.7ms preprocess, 15.5ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 

In [ ]:
print(store) #visualise abit
print(pred_label_indexes_store)

[['Kek Lapis.jpg', 'Kek Lapis'], ['Kuih Kaswi Pandan.jpg', 'Kuih Kaswi Pandan'], ['Kuih Ketayap.jpg', 'Kuih Ketayap'], ['Kuih Lapis.jpg', 'Kuih Lapis'], ['Kuih Seri Muka.png', 'Kuih Seri Muka'], ['Kuih Talam.jpg', 'Kuih Talam'], ['Kuih Ubi Kayu.jpg', 'Kuih Ubi Kayu'], ['Onde-Onde.jpg', 'Onde Onde']]
[5, 4, 7, 1, 6, 0, 2, 3]


In [ ]:
label_indexes = [] #load test_labels data to create ground truth
class_names = []
with open(r"test_folder\test_labels.txt", 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split(',')
        class_name = parts[1].strip().replace('"', '')
        class_name = class_name.replace("-",' ')
        class_names.append(class_name)
        idx = reverse_labels.get(class_name)
        label_indexes.append(idx)

class_names.sort() #sort the data so that it aligns with ultralytics without actually changing the data (does not affect results)
label_indexes_sorted = [reverse_labels.get(name) for name in class_names]

sorted_dict = [reverse_labels.get(name) for name in sorted(class_names)]
sorted_dict.pop(0) #remove the "Class" header
class_names.pop(0)
print(sorted_dict)
print("True labels:", sorted_dict)
print("Corresponding class of true labels", class_names)
print("Predicted labels", pred_label_indexes_store)
print("Files and model predictions", store)

[5, 4, 7, 1, 6, 0, 2, 3]
True labels: [5, 4, 7, 1, 6, 0, 2, 3]
Corresponding class of true labels ['Kek Lapis', 'Kuih Kaswi Pandan', 'Kuih Ketayap', 'Kuih Lapis', 'Kuih Seri Muka', 'Kuih Talam', 'Kuih Ubi Kayu', 'Onde Onde']
Predicted labels [5, 4, 7, 1, 6, 0, 2, 3]
Files and model predictions [['Kek Lapis.jpg', 'Kek Lapis'], ['Kuih Kaswi Pandan.jpg', 'Kuih Kaswi Pandan'], ['Kuih Ketayap.jpg', 'Kuih Ketayap'], ['Kuih Lapis.jpg', 'Kuih Lapis'], ['Kuih Seri Muka.png', 'Kuih Seri Muka'], ['Kuih Talam.jpg', 'Kuih Talam'], ['Kuih Ubi Kayu.jpg', 'Kuih Ubi Kayu'], ['Onde-Onde.jpg', 'Onde Onde']]


In [ ]:
for items in store: #add back hyphen for onde-onde.
    if items[1].replace(" ", "").lower() == "ondeonde":
        items[1] = "Onde-Onde"
for idx, items in enumerate(class_names): #add back hyphen for onde-onde.
    if items.replace(" ", "").lower() == "ondeonde":
        class_names[idx] = "Onde-Onde"
print(class_names)

dataset = [] #create dataset
class_probs = prediction[0].probs.data.tolist()
print(class_probs[0])
for i,items in enumerate(store):
    image = items[0]
    pred_label = items[1]
    pred_index = pred_label_indexes_store[i]
    class_prob = class_probs[i]  # Use your actual probabilities here
    true_index = class_names[i]
    dataset.append([image, pred_index, pred_label, class_prob, true_index])
    
print(dataset)

['Kek Lapis', 'Kuih Kaswi Pandan', 'Kuih Ketayap', 'Kuih Lapis', 'Kuih Seri Muka', 'Kuih Talam', 'Kuih Ubi Kayu', 'Onde-Onde']
1.2191545692985528e-06
[['Kek Lapis.jpg', 5, 'Kek Lapis', 1.2191545692985528e-06, 'Kek Lapis'], ['Kuih Kaswi Pandan.jpg', 4, 'Kuih Kaswi Pandan', 1.478978265367914e-05, 'Kuih Kaswi Pandan'], ['Kuih Ketayap.jpg', 7, 'Kuih Ketayap', 6.33520758128725e-05, 'Kuih Ketayap'], ['Kuih Lapis.jpg', 1, 'Kuih Lapis', 3.992577148892451e-06, 'Kuih Lapis'], ['Kuih Seri Muka.png', 6, 'Kuih Seri Muka', 1.1620790019151173e-06, 'Kuih Seri Muka'], ['Kuih Talam.jpg', 0, 'Kuih Talam', 2.022571834459086e-06, 'Kuih Talam'], ['Kuih Ubi Kayu.jpg', 2, 'Kuih Ubi Kayu', 7.845013669793843e-07, 'Kuih Ubi Kayu'], ['Onde-Onde.jpg', 3, 'Onde-Onde', 0.9999127388000488, 'Onde-Onde']]


In [ ]:
headers = ['image', 'predicted_class_index', 'predicted_label', 'class_probabilities', 'true_class_index']

df = pd.DataFrame(dataset, columns=headers) #make dataframe with columns and index as image.
df.set_index("image") #generate df

,predicted_class_index,predicted_label,class_probabilities,true_class_index
image,,,,
Kek Lapis.jpg,5,Kek Lapis,1.219155e-06,Kek Lapis
Kuih Kaswi Pandan.jpg,4,Kuih Kaswi Pandan,1.478978e-05,Kuih Kaswi Pandan
Kuih Ketayap.jpg,7,Kuih Ketayap,6.335208e-05,Kuih Ketayap
Kuih Lapis.jpg,1,Kuih Lapis,3.992577e-06,Kuih Lapis
Kuih Seri Muka.png,6,Kuih Seri Muka,1.162079e-06,Kuih Seri Muka
Kuih Talam.jpg,0,Kuih Talam,2.022572e-06,Kuih Talam
Kuih Ubi Kayu.jpg,2,Kuih Ubi Kayu,7.845014e-07,Kuih Ubi Kayu
Onde-Onde.jpg,3,Onde-Onde,9.999127e-01,Onde-Onde


In [ ]:
precision = precision_score(sorted_dict, pred_label_indexes_store, average='macro') #report metrics
recall = recall_score(sorted_dict, pred_label_indexes_store, average='macro')
f1 = f1_score(sorted_dict, pred_label_indexes_store, average='macro')

print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1 Score:  {f1:.3f}")

print("\nClassification Report:")
print(classification_report(sorted_dict, pred_label_indexes_store))


Precision: 1.000
Recall:    1.000
F1 Score:  1.000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8

